# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [19]:
#print(school_data_complete.head(5))

# Calculate the total number of students
total_students = len(school_data_complete["Student ID"].unique())

# Calculate the total number of schools
total_schools = len(school_data_complete["school_name"].unique())

          
# Calculate total school budget:
# (a) Group the data frme by schools
# (b) Read school budget for each school in the group and sum them 
school_groups = school_data_complete.groupby("school_name")
total_budget = (school_groups.first()["budget"]).sum()

# Calculate the average math score
average_math_score = school_data_complete["math_score"].mean()

# Calculate the average reading score
average_reading_score = school_data_complete["reading_score"].mean()

# alculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_passing_rate = (average_math_score + average_reading_score) / 2.0

# Calculate the percentage of students with a passing math score (70 or greater)
passing_math_score =  school_data_complete.loc[school_data_complete["math_score"] >= 70, ["math_score"]]
passing_math_score_percent = float((passing_math_score.count()) / total_students)

# Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_score =  school_data_complete.loc[school_data_complete["reading_score"] >= 70, ["reading_score"]]
passing_reading_score_percent = float((passing_reading_score.count()) / total_students)

# Create DataFrame to hold above results
summary_df = pd.DataFrame({ "Total Schools": [total_schools],
                            "Total Students": [total_students],
                            "Total Budget": [total_budget],
                            "Average Math Score": average_math_score,
                            "Average Reading Score": average_reading_score,
                            "% Passing Math": [passing_math_score_percent],
                            "% Passing Reading": [passing_reading_score_percent],
                            "Overall Passing Rate": [overall_passing_rate]
                          })


# Give displayed data cleaner formatting
summary_df["Total Students"] = summary_df["Total Students"].map("{:,}".format)
summary_df["Total Budget"] = summary_df["Total Budget"].astype(float).map("${:,.2f}".format)
summary_df["% Passing Math"] = summary_df["% Passing Math"].map("{:.6%}".format)
summary_df["% Passing Reading"] = summary_df["% Passing Reading"].map("{:.6%}".format)

summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853%,85.805463%,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [21]:
# school_data_complete.head()

# Group data frame by school name
school_groups = school_data_complete.groupby("school_name")

# Create lsit of school names
name_of_the_school = (school_groups["school_name"]).first()

# Total students per school
students_per_school = school_groups["Student ID"].count()

# School type
school_type = school_groups["type"].first()

# Total shcool budget
total_school_budget = school_groups["budget"].first()

# Per student budget
per_student_budget = total_school_budget / students_per_school

# Average math score 
avg_math_score = school_groups["math_score"].mean()

# Average reading score 
avg_reading_score = school_groups["reading_score"].mean()

# Percentage students passing math
students_passing_maths = school_groups["math_score"].apply(lambda x: x[x >= 70].count())
percentage_students_passing_math = students_passing_maths / students_per_school

# Percentage students passing reading
students_passing_reading = school_groups["reading_score"].apply(lambda x: x[x >= 70].count())
percentage_students_passing_reading = students_passing_reading / students_per_school
#percentage_students_passing_reading.head()

# Overall Passing rate
overall_passing_rate = (percentage_students_passing_math + percentage_students_passing_reading ) / 2


# Create DataFrame to hold above results
summary_df = pd.DataFrame({ 
                            "School Type": school_type,
                            "Total Students": students_per_school,
                            "Total School Budget": total_school_budget,
                            "Per Student Budget": per_student_budget,
                            "Average Math Score": avg_math_score,
                            "Average Reading Score": avg_reading_score,
                            "% Passing Math": percentage_students_passing_math,
                            "% Passing Reading": percentage_students_passing_reading,
                            "% Overall Passing Rate": overall_passing_rate 
                         }, index = name_of_the_school)


# Give displayed data cleaner formatting
summary_df["Total School Budget"] = summary_df["Total School Budget"].astype(float).map("${:,.2f}".format)
summary_df["Per Student Budget"] = summary_df["Per Student Budget"].map("{:.2f}".format)
summary_df["% Passing Math"] = summary_df["% Passing Math"] * 100
summary_df["% Passing Reading"] = summary_df["% Passing Reading"] * 100
summary_df["% Overall Passing Rate"] = summary_df["% Overall Passing Rate"] * 100

# Sort and display the top five schools in overall passing rate
top_five_performing_school = (summary_df.sort_values(by=['% Overall Passing Rate'], ascending=False)).head(5)
top_five_performing_school


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [22]:
# Sort and display the five worst-performing schools
worst_five_performing_school = (summary_df.sort_values(by=['% Overall Passing Rate'], ascending=True)).head(5)
worst_five_performing_school


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [23]:
# Data Frame for 9th, 10th, 11th and 12th grade math score
grade_9th = school_data_complete.loc[school_data_complete["grade"] == "9th", ["school_name", "math_score"]]
grade_10th = school_data_complete.loc[school_data_complete["grade"] == "10th", ["school_name", "math_score"]]
grade_11th = school_data_complete.loc[school_data_complete["grade"] == "11th", ["school_name", "math_score"]]
grade_12th = school_data_complete.loc[school_data_complete["grade"] == "12th", ["school_name", "math_score"]]

# Group each math score by school name
grade_9th_mean = (grade_9th.groupby("school_name")).mean()
grade_10th_mean = (grade_10th.groupby("school_name")).mean()
grade_11th_mean = (grade_11th.groupby("school_name")).mean()
grade_12th_mean = (grade_12th.groupby("school_name")).mean()

# Combine all the grades math scores. Rename the columns
summary_df = pd.concat([grade_9th_mean, grade_10th_mean, grade_11th_mean, grade_12th_mean], axis=1)
summary_df.columns = ["9th", "10th", "11th", "12th"]

# Print the table
summary_df


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [24]:
# Data Frame for 9th, 10th, 11th and 12th grade reading score
grade_9th = school_data_complete.loc[school_data_complete["grade"] == "9th", ["school_name", "reading_score"]]
grade_10th = school_data_complete.loc[school_data_complete["grade"] == "10th", ["school_name", "reading_score"]]
grade_11th = school_data_complete.loc[school_data_complete["grade"] == "11th", ["school_name", "reading_score"]]
grade_12th = school_data_complete.loc[school_data_complete["grade"] == "12th", ["school_name", "reading_score"]]

# Group each reading score by school name
grade_9th_mean = (grade_9th.groupby("school_name")).mean()
grade_10th_mean = (grade_10th.groupby("school_name")).mean()
grade_11th_mean = (grade_11th.groupby("school_name")).mean()
grade_12th_mean = (grade_12th.groupby("school_name")).mean()

# Combine all the grades reading score. Rename the columns
summary_df = pd.concat([grade_9th_mean, grade_10th_mean, grade_11th_mean, grade_12th_mean], axis=1)
summary_df.columns = ["9th", "10th", "11th", "12th"]

# Print the table
summary_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [25]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [26]:
# Create spenging column showing group names
school_data_complete["spending"] = pd.cut(school_data_complete["budget"]/school_data_complete["size"], spending_bins, labels = group_names)

# Group by spending. Calculate counts per spending group
spending_groups = school_data_complete.groupby("spending")
spending_groups_count = spending_groups["Student ID"].count()

# Calculate average math and average reading per spending group
avg_math = spending_groups["math_score"].mean()
avg_read = spending_groups["reading_score"].mean()

# Calculate passing math. Group by spending. Count the groups
pass_math = school_data_complete[school_data_complete["math_score"] >= 70]
pass_math_groups = pass_math.groupby("spending")
pass_math_groups_count = pass_math_groups["Student ID"].count()

# Calculate passing reading. Group by spending. Count the groups
pass_reading = school_data_complete[school_data_complete["reading_score"] >= 70]
pass_reading_groups = pass_reading.groupby("spending")
pass_reading_groups_count = pass_reading_groups["Student ID"].count()

# Math pass percentage per group
pass_math_groups_percentage = (pass_math_groups_count / spending_groups_count) * 100

# Reading pass percentage per group
pass_reading_groups_percentage = (pass_reading_groups_count / spending_groups_count) * 100

# Overall pass rate per group
overall_pass = (pass_math_groups_percentage + pass_reading_groups_percentage)/2


# Create data frame to hold above results
summary_df = pd.DataFrame({ "Average Math Score": avg_math,
                            "Average Reading Score": avg_read,
                            "% Passing Math": pass_math_groups_percentage,
                            "% Passing Reading": pass_reading_groups_percentage,
                            "Overall Passing Rate": overall_pass
                          })

summary_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
spending,,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


## Scores by School Size

* Perform the same operations as above, based on school size.

In [28]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [29]:
# Create size_bin column showing group per school size
school_data_complete["size_bins"] = pd.cut(school_data_complete["size"], size_bins, labels = group_names)

# Group by school size. Calculate counts per group
size_groups = school_data_complete.groupby("size_bins")
size_groups_count = size_groups["Student ID"].count()

# Calculate average math and average reading per group
avg_math = size_groups["math_score"].mean()
avg_read = size_groups["reading_score"].mean()

# Calculate passing math. Group by school size. Count the groups
pass_math = school_data_complete[school_data_complete["math_score"] >= 70]
pass_math_groups = pass_math.groupby("size_bins")
pass_math_groups_count = pass_math_groups["Student ID"].count()

# Calculate passing reading. Group by school size. Count the groups
pass_reading = school_data_complete[school_data_complete["reading_score"] >= 70]
pass_reading_groups = pass_reading.groupby("size_bins")
pass_reading_groups_count = pass_reading_groups["Student ID"].count()

# Math pass percentage per group
pass_math_groups_percentage = (pass_math_groups_count / size_groups_count) * 100

# Reading pass percentage per group
pass_reading_groups_percentage = (pass_reading_groups_count / size_groups_count) * 100

# Overall pass rate per group
overall_pass = (pass_math_groups_percentage + pass_reading_groups_percentage)/2


# Create data frame to hold above results
summary_df = pd.DataFrame({ "Average Math Score": avg_math,
                            "Average Reading Score": avg_read,
                            "% Passing Math": pass_math_groups_percentage,
                            "% Passing Reading": pass_reading_groups_percentage,
                            "Overall Passing Rate": overall_pass
                          })

summary_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
size_bins,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type.

In [34]:
# Group by school type. Calculate counts per group
type_groups = school_data_complete.groupby("type")
type_groups_count = type_groups["Student ID"].count()

# Calculate average math and average reading per group
avg_math = type_groups["math_score"].mean()
avg_read = type_groups["reading_score"].mean()

# Calculate passing math. Group by school type. Count the groups
pass_math = school_data_complete[school_data_complete["math_score"] >= 70]
pass_math_groups = pass_math.groupby("type")
pass_math_groups_count = pass_math_groups["Student ID"].count()

# Calculate passing reading. Group by school type. Count the groups
pass_reading = school_data_complete[school_data_complete["reading_score"] >= 70]
pass_reading_groups = pass_reading.groupby("type")
pass_reading_groups_count = pass_reading_groups["Student ID"].count()

# Math pass percentage per group
pass_math_groups_percentage = (pass_math_groups_count / type_groups_count) * 100

# Reading pass percentage per group
pass_reading_groups_percentage = (pass_reading_groups_count / type_groups_count) * 100

# Overall pass rate per group
overall_pass = (pass_math_groups_percentage + pass_reading_groups_percentage)/2


# Create data frame to hold above results
summary_df = pd.DataFrame({ "Average Math Score": avg_math,
                            "Average Reading Score": avg_read,
                            "% Passing Math": pass_math_groups_percentage,
                            "% Passing Reading": pass_reading_groups_percentage,
                            "Overall Passing Rate": overall_pass
                          })

summary_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
